In [2]:
#! wget https://data.virginia.gov/dataset/de833a43-7019-444c-8384-9e0cf5255140/resource/60506bbb-685f-4360-8a8c-30e137ce3615/download/community-policing-data.csv

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv("community-policing-data.csv", on_bad_lines="skip")
terry_stops = data[data["REASON FOR STOP"] == "Terry Stop"]

C:\Users\sophi\AppData\Local\Temp\ipykernel_18632\4050143798.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("community-policing-data.csv", on_bad_lines="skip")


In [5]:
# replace - with NaN
feature_list = ["FORCE USED BY SUBJECT", "FORCE USED BY OFFICER", "VEHICLE SEARCHED", "PERSON SEARCHED", "ENGLISH SPEAKING"]
for feature in feature_list:
  terry_stops[feature].replace('-', np.nan, inplace= True)


C:\Users\sophi\AppData\Local\Temp\ipykernel_18632\3029540058.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  terry_stops[feature].replace('-', np.nan, inplace= True)
C:\Users\sophi\AppData\Local\Temp\ipykernel_18632\3029540058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  terry_stops[feature].replace('-', np.nan, inplace= True)


In [6]:
# removing all the columns with all null values
terry_stops = terry_stops.drop(columns = ["VIOLATION TYPE", "ADDITIONAL ARREST", "VIRGINIA CRIME CODE"])

# the features without null values are:
# stop date, agency name, jurisdiction, reason for stop, person type, race, ethnicity, age, gender, action taken, residency

In [7]:
# I am going to remove reason for stop because they are all the same
terry_stops = terry_stops.drop(columns = ["REASON FOR STOP"])

# I am going to remove this for the clustering because it is not helpful
# We can later do it by month or year if we are intrested but not for now
terry_stops = terry_stops.drop(columns = ["STOP_DATE"])


# I am dropping these because I think jurisdiction cover the same info, but its less grandular so might led to fewer generalizations
terry_stops = terry_stops.drop(columns = ["AGENCY NAME",	"LOCATION"])

In [8]:
terry_stops = terry_stops[terry_stops['FORCE USED BY OFFICER'].notna()]
terry_stops['FORCE USED BY OFFICER'].replace('Y', 1, inplace= True)
terry_stops['FORCE USED BY OFFICER'].replace('N', 0, inplace=True)

Y = terry_stops['FORCE USED BY OFFICER']
X = terry_stops.drop('FORCE USED BY OFFICER', axis=1)

C:\Users\sophi\AppData\Local\Temp\ipykernel_18632\3713651789.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  terry_stops['FORCE USED BY OFFICER'].replace('Y', 1, inplace= True)
C:\Users\sophi\AppData\Local\Temp\ipykernel_18632\3713651789.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

In [9]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler())
    ])


num_attribs = list(X.select_dtypes(include=[np.number]))
# These are the categorical values that I thought would be most useful
cat_few = ["PERSON TYPE","RACE","ETHNICITY", "GENDER", "RESIDENCY", "ACTION TAKEN"]
cat_binary = ["PERSON SEARCHED","VEHICLE SEARCHED",	"FORCE USED BY SUBJECT", "ENGLISH SPEAKING"]
cat_alot = ["JURISDICTION"]
encoder = LabelEncoder()

def binary_map(X):
    return np.where(X == "Y", 1, 0)
binary_encoder = FunctionTransformer(binary_map)

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("binary", binary_encoder, cat_binary),
        ("cat_few", OneHotEncoder(handle_unknown='ignore'), cat_few),
        ("cat_alot", OrdinalEncoder(), cat_alot)
    ])

X = full_pipeline.fit_transform(X)
feature_names = (
    list(num_attribs) +
    list(cat_binary) +
    list(full_pipeline.named_transformers_["cat_few"].get_feature_names_out(cat_few)) +
    list(cat_alot)
)
print(feature_names)

print(X.shape)

['PERSON SEARCHED', 'VEHICLE SEARCHED', 'FORCE USED BY SUBJECT', 'ENGLISH SPEAKING', 'PERSON TYPE_Driver', 'PERSON TYPE_Missing', 'PERSON TYPE_Other Individual', 'PERSON TYPE_Passenger', 'RACE_American Indian or Alaska Native', 'RACE_Asian or Native Hawaiian or Other Pacific Islander', 'RACE_Black or African American', 'RACE_Missing', 'RACE_Unknown', 'RACE_White', 'ETHNICITY_Hispanic or Latino', 'ETHNICITY_Non-Hispanic or Latino', 'ETHNICITY_Unknown', 'GENDER_Female', 'GENDER_Male', 'GENDER_Missing', 'GENDER_Other', 'GENDER_Unknown', 'RESIDENCY_Missing', 'RESIDENCY_Other Virginia jurisdiction resident', 'RESIDENCY_Out of State resident', 'RESIDENCY_Resident of city/county of stop', 'RESIDENCY_Unknown', 'ACTION TAKEN_Arrest', 'ACTION TAKEN_Citation/Summons', 'ACTION TAKEN_Missing', 'ACTION TAKEN_No Enforcement Action', 'ACTION TAKEN_Warning Issued', 'JURISDICTION']
(30858, 33)


# Predicting whether force is used

## Using all characteristics for now

In [20]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score, precision_recall_curve, recall_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### Random Forest

In [22]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
classification_report(y_pred=y_pred, y_true=y_test)
cnf = confusion_matrix(y_pred=y_pred, y_true=y_test)
print(cnf)

[[6068   19]
 [  45   40]]
